## OMPT Start

There are three steps an OpenMP implementation takes to activate a tool. This section explains how the tool and an OpenMP implementation interact to accomplish tool activation.

Step 1. *Determine Whether to Initialize*

A tool is activated by the OMPT interface when it returns a non-NULL pointer to an __ompt_start_tool_result_t__ structure on a call to __ompt_start_tool__ by the OpenMP implementation. There are three ways that a tool can provide a definition of __ompt_start_tool__ to an OpenMP implementation: (1) Statically linking the tool's definition of __ompt_start_tool__ into an OpenMP application. (2) Introducing a dynamically linked library that includes the tool's definition of __ompt_start_tool__ into the application's address space. (3) Providing the name of a dynamically linked library appropriate for the architecture and operating system used by the application in the  _tool-libraries-var_  ICV.

Step 2. *Initializing a First-Party tool*

If a tool-provided implementation of __ompt_start_tool__ returns a non-NULL pointer to an __ompt_start_tool_result_t__ structure, the OpenMP implementation will invoke the tool initializer specified in this structure prior to the occurrence of any OpenMP event.

Step 3. *Monitoring Activity on the Host*

To monitor execution of an OpenMP program on the host device, a tool's initializer must register to receive notification of events that occur as an OpenMP program executes. A tool can register callbacks for OpenMP events using the runtime entry point known as __ompt_set_callback__, which has the following possible return codes: \hfill reak  __ompt_set_error__,  __ompt_set_never__,  __ompt_set_impossible__,  __ompt_set_sometimes__,  __ompt_set_sometimes_paired__,  __ompt_set_always__.

If the __ompt_set_callback__ runtime entry point is called outside a tool's initializer, registration of supported callbacks may fail with a return code of __ompt_set_error__. All callbacks registered with __ompt_set_callback__ or returned by __ompt_get_callback__ use the dummy type signature __ompt_callback_t__. While this is a compromise, it is better than providing unique runtime entry points with precise type signatures to set and get the callback for each unique runtime entry point type signature.

----------------

To use the OMPT interface a tool must provide a globally-visible implementation of the __ompt_start_tool__ function. The function returns a pointer to an __ompt_start_tool_result_t__ structure  that contains callback pointers for tool initialization and finalization as well  as a data word,  _tool_data_ , that is to be passed by reference to these callbacks. A __NULL__ return indicates the tool will not use the OMPT interface. The runtime execution of __ompt_start_tool__ is triggered by the first OpenMP  directive or OpenMP API routine call.

In the example below, the user-provided __ompt_start_tool__ function performs a check to make sure the runtime OpenMP version that OMPT supports  (provided by the  *omp_version*  argument) is identical to the  OpenMP implementation (compile-time) version. Also, a __NULL__ is returned to indicate that the OMPT interface is not used (no callbacks and tool data are specified).

**Note**: The  *omp-tools.h*  file is included.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       ompt_start.1
* type:       C
* version:    omp_5.0
*/
#include <stdio.h>
#include <omp.h>
#include <omp-tools.h>

ompt_start_tool_result_t *ompt_start_tool(
unsigned int omp_version,
const char *runtime_version
){
  if(omp_version != _OPENMP)
    printf("Warning: OpenMP runtime version (%i) "
           "does not match the compile time version (%i)"
           " for runtime identifying as %s\n",
           omp_version, _OPENMP, runtime_version);
  // Returning NULL will disable this as an OMPT tool,
  // allowing other tools to be loaded
  return NULL;
}

int main(void){
  printf("Running with %i threads\n", omp_get_max_threads());
  return 0;
}